In [317]:
import pandas as pd

# Creating the dataframe used for display purposes 

In [318]:
dfd = pd.read_csv('all_seasons.csv',index_col=0)
dfd.reset_index(inplace=True, drop = True)
dfd['comp'].unique()

array(['Premier League', 'EFL Cup', 'Champions Lg', 'FA Cup', 'La Liga',
       'Supercopa de España', 'Copa del Rey', 'Ligue 1',
       'Trophée des Champions', 'Coupe de France', 'Bundesliga',
       'Super Cup', 'DFL-Supercup', 'DFB-Pokal', 'Community Shield',
       'Primeira Liga', 'Serie A', 'Coppa Italia', 'Supercoppa Italiana',
       'Europa Lg', 'First Division A', 'Eredivisie', 'Super League',
       'Süper Lig', 'Superliga', 'NB I', 'Swiss Super League',
       'Allsvenskan', '1. HNL', 'Premiership', 'First League'],
      dtype=object)

In [319]:
#dfd = dfd[dfd['comp']=='Champions Lg']
dfd = dfd[dfd['season']=='2022-2023']
dfd.shape

(1070, 236)

In [320]:
len(dfd['team'].unique())

32

In [321]:
cols_todel = ['ga','match report_x','match report_y','date_opp','time_opp','comp_opp','round_opp','day_opp','venue_opp',
              'result_opp','ga_opp','opponent_opp','xg', 'xg_y', 'match report_opp','poss_y', 'captain','referee','notes', 'attendance', 'formation']

In [322]:
dfd.drop(cols_todel, axis=1, inplace=True)
dfd.loc[dfd["gf"].str.contains(" "),'gf'] = dfd["gf"].str[0] # if the Score column starts with "(", then grab the first from left char.
dfd.loc[dfd["gf_opp"].str.contains(" "),'gf_opp'] = dfd["gf_opp"].str[0]
dfd['opponent'] = dfd['opponent'].apply(lambda x: x[3:].lstrip() if x[0].islower() else x)  # If the format of the name starts with small letters, that is the country code. 
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Atletico Madrid' if x == 'Atlético Madrid' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Bayer Leverkusen' if x == 'Leverkusen' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Eintracht Frankfurt' if x == 'Eint Frankfurt' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Internazionale' if x == 'Inter' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Paris Saint Germain' if x == 'Paris S-G' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Red Bull Salzburg' if x == 'RB Salzburg' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Shakhtar Donetsk' if x == 'Shakhtar' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Tottenham Hotspur' if x == 'Tottenham' else x)
dfd['opponent'] = dfd['opponent'].apply(lambda x:'Viktoria Plzen' if x == 'Viktoria Plzeň' else x)

dfd['date'] = pd.to_datetime(dfd['date'],dayfirst=True)
dfd["venue_code"] = dfd["venue"].astype("category").cat.codes
dfd["opp_code"] = dfd["opponent"].astype("category").cat.codes #
dfd["hour"] = dfd["time"].str.replace(":.+", "", regex=True).astype("int")
dfd["day_code"] = dfd["date"].dt.dayofweek


In [323]:
dfd.shape

(1070, 219)

In [324]:
dfd.head()

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,recov_opp,won_opp,won%_opp,xga,team,season,venue_code,opp_code,hour,day_code
3025,2022-07-30,20:30,DFL-Supercup,DFL-Supercup,Sat,Away,W,5,RB Leipzig,5.0,...,NaN,NaN,NaN,NaN,Bayern Munich,2022-2023,0,180,20,5
3026,2022-08-05,20:30,Bundesliga,Matchweek 1,Fri,Away,W,6,Eintracht Frankfurt,6.0,...,59.0,6.0,66.7,0.9,Bayern Munich,2022-2023,0,65,20,4
3027,2022-08-14,17:30,Bundesliga,Matchweek 2,Sun,Home,W,2,Wolfsburg,2.0,...,42.0,14.0,58.3,0.5,Bayern Munich,2022-2023,1,262,17,6
3028,2022-08-21,17:30,Bundesliga,Matchweek 3,Sun,Away,W,7,Bochum,6.0,...,47.0,14.0,60.9,0.7,Bayern Munich,2022-2023,0,31,17,6
3029,2022-08-27,18:30,Bundesliga,Matchweek 4,Sat,Home,D,1,M'Gladbach,1.0,...,38.0,7.0,36.8,0.5,Bayern Munich,2022-2023,1,139,18,5


In [325]:
len(dfd['team'].unique())

32

In [326]:
dfd[dfd['team']=='Celtic']

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,recov_opp,won_opp,won%_opp,xga,team,season,venue_code,opp_code,hour,day_code
3991,2022-07-31,16:30,Premiership,Regular season,Sun,Home,W,2,Aberdeen,2.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,1,5,16,6
3992,2022-08-06,15:00,Premiership,Regular season,Sat,Away,W,3,Ross County,3.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,0,196,15,5
3993,2022-08-14,12:00,Premiership,Regular season,Sun,Away,W,5,Kilmarnock,5.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,0,119,12,6
3994,2022-08-21,15:00,Premiership,Regular season,Sun,Home,W,2,Hearts,2.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,1,104,15,6
3995,2022-08-28,12:00,Premiership,Regular season,Sun,Away,W,9,Dundee United,9.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,0,63,12,6
3996,2022-09-03,12:30,Premiership,Regular season,Sat,Home,W,4,Rangers,4.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,1,183,12,5
3997,2022-09-06,20:00,Champions Lg,Group stage,Tue,Home,L,0,Real Madrid,0.0,...,46.0,5.0,62.5,1.9,Celtic,2022-2023,1,186,20,1
3998,2022-09-14,19:45,Champions Lg,Group stage,Wed,Away,D,1,Shakhtar Donetsk,0.0,...,42.0,6.0,50.0,0.4,Celtic,2022-2023,0,209,19,2
3999,2022-09-18,12:00,Premiership,Regular season,Sun,Away,L,0,St Mirren,0.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,0,226,12,6
4000,2022-10-01,15:00,Premiership,Regular season,Sat,Home,W,2,Motherwell,2.0,...,NaN,NaN,NaN,NaN,Celtic,2022-2023,1,155,15,5


In [327]:
# missing = dfd.isna().mean(axis =0)
# missing[missing>0] # Many rows still have null values. Delete them. 

In [328]:
dfd = dfd.dropna(axis = 0)
dfd.shape

(611, 219)

In [329]:
len(dfd['team'].unique())

32

In [330]:
dfd['team'].unique()

array(['Bayern Munich', 'Benfica', 'Napoli', 'Real Madrid', 'Chelsea',
       'Manchester City', 'Liverpool', 'Paris Saint Germain', 'Milan',
       'RB Leipzig', 'Internazionale', 'Porto', 'Dortmund',
       'Tottenham Hotspur', 'Club Brugge', 'Eintracht Frankfurt',
       'Barcelona', 'Sporting CP', 'Marseille', 'Shakhtar Donetsk',
       'Red Bull Salzburg', 'Ajax', 'Atletico Madrid', 'Bayer Leverkusen',
       'Sevilla', 'Dinamo Zagreb', 'Juventus', 'FC Copenhagen',
       'Maccabi Haifa', 'Celtic', 'Viktoria Plzen', 'Rangers'],
      dtype=object)

In [331]:
dfd['opponent'].unique()

array(['Eintracht Frankfurt', 'Wolfsburg', 'Bochum', "M'Gladbach",
       'Union Berlin', 'Stuttgart', 'Augsburg', 'Bayer Leverkusen',
       'Dortmund', 'Viktoria Plzen', 'Hoffenheim', 'Mainz 05',
       'Hertha BSC', 'Werder Bremen', 'Schalke 04', 'RB Leipzig', 'Köln',
       'Paris Saint Germain', 'Arouca', 'Casa Pia', 'Paços', 'Vizela',
       'Maccabi Haifa', 'Famalicão', 'Marítimo', 'Vitória', 'Rio Ave',
       'Porto', 'Juventus', 'Braga', 'Sporting CP', 'Club Brugge',
       'Hellas Verona', 'Fiorentina', 'Liverpool', 'Milan', 'Torino',
       'Ajax', 'Cremonese', 'Bologna', 'Rangers', 'Sassuolo', 'Atalanta',
       'Empoli', 'Internazionale', 'Sampdoria', 'Salernitana', 'Almería',
       'Celta Vigo', 'Espanyol', 'Betis', 'Celtic', 'Mallorca',
       'Atletico Madrid', 'Osasuna', 'Shakhtar Donetsk', 'Getafe',
       'Sevilla', 'Girona', 'Rayo Vallecano', 'Cádiz', 'Valladolid',
       'Villarreal', 'Athletic Club', 'Everton', 'Tottenham Hotspur',
       'Leeds United', 'Leicest

In [332]:
dfd.sort_values(by='date').reset_index(inplace=True)
num_cols = dfd.select_dtypes('number').columns
num_cols.has_duplicates

False

In [333]:
new_cols = [f"{c}_rolling" for c in list(num_cols)] # add_rolling to column names
new_cols

['gls_rolling',
 'sh_rolling',
 'sot_rolling',
 'sot%_rolling',
 'g/sh_rolling',
 'g/sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling',
 'xg_x_rolling',
 'npxg_rolling',
 'npxg/sh_rolling',
 'g-xg_rolling',
 'np:g-xg_rolling',
 'gls_opp_rolling',
 'sh_opp_rolling',
 'sot_opp_rolling',
 'sot%_opp_rolling',
 'g/sh_opp_rolling',
 'g/sot_opp_rolling',
 'dist_opp_rolling',
 'fk_opp_rolling',
 'pk_opp_rolling',
 'pkatt_opp_rolling',
 'xg_opp_rolling',
 'npxg_opp_rolling',
 'npxg/sh_opp_rolling',
 'g-xg_opp_rolling',
 'np:g-xg_opp_rolling',
 'sota_rolling',
 'saves_rolling',
 'save%_rolling',
 'cs_rolling',
 'psxg_rolling',
 'psxg+/-_rolling',
 'pka_rolling',
 'pksv_rolling',
 'pkm_rolling',
 'cmp_rolling',
 'att_rolling',
 'cmp%_rolling',
 'thr_rolling',
 'launch%_rolling',
 'avglen_rolling',
 'opp_rolling',
 'stp_rolling',
 'stp%_rolling',
 '#opa_rolling',
 'avgdist_rolling',
 'sota_opp_rolling',
 'saves_opp_rolling',
 'save%_opp_rolling',
 'cs_opp_rolling',
 'ps

In [334]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3,closed="left").mean() # closed = left -> to avoid data leakage from future matches
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [335]:
dfd_rolling = dfd.groupby('team').apply(lambda x: rolling_averages(x,num_cols,new_cols))
dfd_rolling = dfd_rolling.droplevel('team')

C:\Users\Ramin\AppData\Local\Temp\ipykernel_16772\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group[new_cols] = rolling_stats
C:\Users\Ramin\AppData\Local\Temp\ipykernel_16772\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  group[new_cols] = rolling_stats
C:\Users\Ramin\AppData\Local\Temp\ipykernel_16772\1506766940.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns a

In [336]:
dfd_rolling['team'].unique()

array(['Ajax', 'Atletico Madrid', 'Barcelona', 'Bayer Leverkusen',
       'Bayern Munich', 'Benfica', 'Celtic', 'Chelsea', 'Club Brugge',
       'Dinamo Zagreb', 'Dortmund', 'Eintracht Frankfurt',
       'FC Copenhagen', 'Internazionale', 'Juventus', 'Liverpool',
       'Maccabi Haifa', 'Manchester City', 'Marseille', 'Milan', 'Napoli',
       'Paris Saint Germain', 'Porto', 'RB Leipzig', 'Rangers',
       'Real Madrid', 'Red Bull Salzburg', 'Sevilla', 'Shakhtar Donetsk',
       'Sporting CP', 'Tottenham Hotspur', 'Viktoria Plzen'], dtype=object)

In [337]:
b = sorted(list(dfd['team'].unique()))

In [338]:
c = sorted(list(dfd_rolling['team'].unique()))

In [339]:
len(dfd_rolling['team'].unique())

32

In [340]:
len(dfd_rolling[dfd_rolling['season']=='2022-2023']['team'].unique())

32

In [341]:
dfd_rolling[dfd_rolling['team']=='Barcelona']

,date,time,comp,round,day,venue,result,gf,opponent,gls,...,pkcon_opp_rolling,og_opp_rolling,recov_opp_rolling,won_opp_rolling,won%_opp_rolling,xga_rolling,venue_code_rolling,opp_code_rolling,hour_rolling,day_code_rolling
3592,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,Sevilla,3.0,...,0.000000,0.0,47.000000,7.333333,42.966667,0.700000,0.666667,205.333333,20.666667,5.666667
3593,2022-09-07,21:00,Champions Lg,Group stage,Wed,Home,W,5,Viktoria Plzen,5.0,...,0.000000,0.0,47.333333,7.000000,44.366667,0.866667,0.333333,213.000000,20.666667,5.666667
3599,2022-10-09,21:00,La Liga,Matchweek 8,Sun,Home,W,1,Celta Vigo,1.0,...,0.000000,0.0,48.333333,7.333333,42.400000,0.866667,0.666667,234.000000,20.333333,4.333333
3600,2022-10-12,21:00,Champions Lg,Group stage,Wed,Home,D,3,Internazionale,3.0,...,0.000000,0.0,49.666667,9.000000,45.733333,1.100000,0.666667,168.000000,21.000000,4.333333
3602,2022-10-20,21:00,La Liga,Matchweek 10,Thu,Home,W,3,Villarreal,3.0,...,0.000000,0.0,48.000000,13.333333,46.466667,1.433333,1.000000,136.666667,21.000000,3.333333
3603,2022-10-23,21:00,La Liga,Matchweek 11,Sun,Home,W,4,Athletic Club,4.0,...,0.000000,0.0,43.000000,11.333333,40.666667,1.300000,1.000000,137.000000,21.000000,3.666667
3605,2022-10-29,21:00,La Liga,Matchweek 12,Sat,Away,W,1,Valencia,1.0,...,0.000000,0.0,43.000000,9.333333,41.300000,0.966667,1.000000,128.000000,21.000000,3.666667
3606,2022-11-01,21:00,Champions Lg,Group stage,Tue,Away,W,4,Viktoria Plzen,4.0,...,0.000000,0.0,43.000000,4.666667,43.333333,0.366667,0.666667,171.000000,21.000000,4.666667
3607,2022-11-05,21:00,La Liga,Matchweek 13,Sat,Home,W,2,Almería,2.0,...,0.000000,0.0,49.000000,8.333333,56.666667,1.100000,0.333333,170.666667,21.000000,4.000000
3608,2022-11-08,21:30,La Liga,Matchweek 14,Tue,Away,W,2,Osasuna,2.0,...,0.333333,0.0,48.333333,8.666667,54.033333,1.100000,0.333333,167.333333,21.000000,3.666667


## Minmax normaliation for the radar plots. 

In [342]:
dfd_rolling[new_cols] = (dfd_rolling[new_cols]-dfd_rolling[new_cols].min())/(dfd_rolling[new_cols].max()-dfd_rolling[new_cols].min())

In [343]:
dfd_rolling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 3748 to 4048
Columns: 426 entries, date to day_code_rolling
dtypes: datetime64[ns](1), float64(408), int16(1), int32(1), int64(3), int8(1), object(11)
memory usage: 1.7+ MB


In [344]:
dfd_rolling.to_csv('dfd_rolling.csv')


In [349]:
dfd_rolling['season']

3748    2022-2023
3750    2022-2023
3752    2022-2023
3753    2022-2023
3754    2022-2023
          ...    
3515    2022-2023
3516    2022-2023
3518    2022-2023
4046    2022-2023
4048    2022-2023
Name: season, Length: 515, dtype: object

In [345]:
len(dfd_rolling['team'].unique())

32

#### Useful stuff
Do the following to get a list of the numeric columns, or select an type of column based on its tpes. 

```[key for key in dict(DF.dtypes) if dict(DF.dtypes)[key] in ['float64', 'int64']]```

predictors2 = [b for a, b in zip(sfs.get_support(), all_cols) if a] # extracting the features sfs has chosen for us. all_cols is a string list of all the columns, this uses the sfs.get_support mask.

In [346]:
dfd_rolling['season'].unique()

array(['2022-2023'], dtype=object)

In [347]:
list(dfd_rolling.columns)

['date',
 'time',
 'comp',
 'round',
 'day',
 'venue',
 'result',
 'gf',
 'opponent',
 'gls',
 'sh',
 'sot',
 'sot%',
 'g/sh',
 'g/sot',
 'dist',
 'fk',
 'pk',
 'pkatt',
 'xg_x',
 'npxg',
 'npxg/sh',
 'g-xg',
 'np:g-xg',
 'gf_opp',
 'gls_opp',
 'sh_opp',
 'sot_opp',
 'sot%_opp',
 'g/sh_opp',
 'g/sot_opp',
 'dist_opp',
 'fk_opp',
 'pk_opp',
 'pkatt_opp',
 'xg_opp',
 'npxg_opp',
 'npxg/sh_opp',
 'g-xg_opp',
 'np:g-xg_opp',
 'sota',
 'saves',
 'save%',
 'cs',
 'psxg',
 'psxg+/-',
 'pka',
 'pksv',
 'pkm',
 'cmp',
 'att',
 'cmp%',
 'thr',
 'launch%',
 'avglen',
 'opp',
 'stp',
 'stp%',
 '#opa',
 'avgdist',
 'sota_opp',
 'saves_opp',
 'save%_opp',
 'cs_opp',
 'psxg_opp',
 'psxg+/-_opp',
 'pka_opp',
 'pksv_opp',
 'pkm_opp',
 'cmp_opp',
 'att_opp',
 'cmp%_opp',
 'thr_opp',
 'launch%_opp',
 'avglen_opp',
 'opp_opp',
 'stp_opp',
 'stp%_opp',
 '#opa_opp',
 'avgdist_opp',
 'totdist',
 'prgdist',
 'ast',
 'xag',
 'xa',
 'kp',
 '1-Mar',
 'ppa',
 'crspa',
 'prgp',
 'totdist_opp',
 'prgdist_opp',
 'as